In [1]:
import os 
import numpy as np
import pickle 
from astropy.table import Table

In [2]:
train_data_head_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/train_final_master_HEAD.FITS'
train_data_phot_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/train_final_master_PHOT.FITS'
test_data_head_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/test_final_master_HEAD.FITS'
test_data_phot_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/test_final_master_PHOT.FITS'

In [3]:
df_header = Table.read(train_data_head_path, format='fits').to_pandas()
df_phot = Table.read(train_data_phot_path, format='fits').to_pandas()

In [4]:
from kndetect.utils import load_pcs, get_event_type, get_data_dir_path
from kndetect.predict_features import extract_features_all_lightcurves
from kndetect.training import append_y_true_col

In [5]:
pcs = load_pcs()

In [ ]:
train_features_df = extract_features_all_lightcurves(df_phot, 
                                                     "SNID", 
                                                     pcs, 
                                                     [b'g', b'r'])

 12%|█▏        | 2692/22280 [05:31<36:53,  8.85it/s]  

In [ ]:
train_features_df = append_y_true_col(features_df=train_features_df,
                                      prediction_type_nos=[149, 150, 151],
                                      meta_df = df_header, 
                                      meta_key_col_name = "SNID",
                                      meta_type_col_name = "SNTYPE")

# Now Train the classifier

In [ ]:
from kndetect.training import train_classifier

In [ ]:
clf, features_df = train_classifier(train_features_df)

In [ ]:
data_dir = get_data_dir_path()
with open(os.path.join(data_dir, 'complete.pkl'), 'wb') as files:
    pickle.dump(clf, files)

In [ ]:
# Generate Test features and Predict probabilities

In [ ]:
data_dir_path = get_data_dir_path()

In [ ]:

df_header_test = Table.read(test_data_head_path, format='fits').to_pandas()
df_phot_test = Table.read(test_data_phot_path, format='fits').to_pandas()

In [ ]:
test_features_df = extract_features_all_lightcurves(df_phot_test, 
                                                     "SNID", 
                                                     pcs, 
                                                     [b'g', b'r'])
test_features_df = append_y_true_col(features_df=test_features_df,
                                      prediction_type_nos=[149, 150, 151],
                                      meta_df = df_header_test, 
                                      meta_key_col_name = "SNID",
                                      meta_type_col_name = "SNTYPE")

In [ ]:
from kndetect.predict import load_classifier, predict_kn_score

In [ ]:
clf1=load_classifier("complete.pkl")

In [ ]:
probabilities, filtered_indices = predict_kn_score(clf1, test_features_df)

In [ ]:
test_features_df['y_pred_score'] = probabilities.T[1]

In [ ]:
test_features_df